In [1]:
import math
from torch import empty

from generate_data import generate_data
import functional as F

In [2]:
N = 1000

train_input, train_target, test_input, test_target = generate_data(1000, True)

print(train_input.shape)
print(train_input.std())
print(train_input.mean())

torch.Size([1000, 2])
tensor(1.0000)
tensor(-1.2785e-07)


In [12]:
F.tanh(train_input)

tensor([[ 0.4290,  0.8069],
        [ 0.8570,  0.1460],
        [-0.7375,  0.8968],
        ...,
        [ 0.7134, -0.8416],
        [-0.9193, -0.6799],
        [ 0.8272,  0.9327]])